In [1]:
import Neilyst

data_1h = Neilyst.get_klines('BTC/USDT', '2023-01-01T00:00:00Z', '2023-12-30T00:00:00Z', timeframe='1h')
data_30m = Neilyst.get_klines('BTC/USDT', '2023-01-01T00:00:00Z', '2023-12-30T00:00:00Z', timeframe='30m')
data_15m = Neilyst.get_klines('BTC/USDT', '2023-01-01T00:00:00Z', '2023-12-30T00:00:00Z', timeframe='15m')
data_5m = Neilyst.get_klines('BTC/USDT', '2023-01-01T00:00:00Z', '2023-12-30T00:00:00Z', timeframe='5m')
data_1m = Neilyst.get_klines('BTC/USDT', '2023-01-01T00:00:00Z', '2023-12-30T00:00:00Z', timeframe='1m')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Data for 2023-01-01-00:00 to 2023-01-01-23:30 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/30m/2023-01-01-00:00 - 2023-01-01-23:30.csv
Data for 2023-01-02-00:00 to 2023-01-02-23:30 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/30m/2023-01-02-00:00 - 2023-01-02-23:30.csv
Data for 2023-01-03-00:00 to 2023-01-03-23:30 saved to /Users/neil/Code/Neilyst/data/binanceusdm-BTC/30m/2023-01-03-00:00 - 2023-01-03-23:30.csv
Data for 2023-01-04-00:00 to 2023-01-04-23:30 saved to /Users/nei

In [2]:
# 计算指标

indicators_1h = Neilyst.get_indicators(data_1h, 'rsi', 'sma20', 'ema9')
indicators_30m = Neilyst.get_indicators(data_30m, 'rsi', 'sma20', 'ema9')
indicators_15m = Neilyst.get_indicators(data_15m, 'rsi', 'sma20', 'ema9')
indicators_5m= Neilyst.get_indicators(data_5m, 'rsi', 'sma20', 'ema9')
indicators_1m= Neilyst.get_indicators(data_1m, 'rsi', 'sma20', 'ema9')

In [ ]:
class MultiSignalStrategy(Neilyst.Strategy):
    def __init__(self, total_balance, trading_fee_ratio, slippage_ratio, data, indicators):
        super().__init__(total_balance, trading_fee_ratio, slippage_ratio, data, indicators)
    
    def run(self, date, price_row, current_pos, current_balance):
        recent_data_15m = self.get_recent_data(date, 2)
        if len(recent_data_15m) >= 2:
            ema_15 = indicators_15m.iloc[0]['ema9']
            ma_15 = indicators_15m.iloc[0]['sma20']

            prev_ema_15 = indicators_15m.iloc[-1]['ema9']
            prev_ma_15 = indicators_15m.iloc[-1]['sma20']

            if current_pos.amount > 0:
                # 此时有仓位，考虑平仓过程
                pass
            if (ema_15 > ma_15) and (prev_ema_15 < prev_ma_15):
                pass

    def pos_management(self):
        pass

In [3]:
class MovingAverageStrategy(Neilyst.Strategy):
    def __init__(self, total_balance, trading_fee_ratio, slippage_ratio, data, indicators):
        super().__init__(total_balance, trading_fee_ratio, slippage_ratio, data, indicators)
        self.short_window = 20
        self.long_window = 60

    def run(self, date, price_row, current_pos, current_balance):
        recent_data = self.get_recent_data(date, 2)

        short_mavg = recent_data['sma20'].head(1)
        long_mavg = recent_data['sma60'].head(1)

        prev_short_mavg = recent_data['sma20'].tail(1)
        prev_long_mavg = recent_data['sma60'].tail(1)

        if short_mavg > long_mavg and prev_short_mavg < prev_long_mavg:
            signal = Neilyst.Signal('long', price_row['close'], 1)
        elif short_mavg < long_mavg and prev_short_mavg > prev_long_mavg:
            if current_pos.amount > 0:
                signal = Neilyst.Signal('close', price_row['close'], current_pos.amount)
            else:
                signal = None
        else:
            signal = None
        
        return signal

strategy = MovingAverageStrategy(50000, 0, 0, data, indicators)
result = Neilyst.backtest('BTC/USDT', '2023-01-01T00:00:00Z', '2023-12-30T00:00:00Z', strategy)
result

2023-01-01T00:00:00Z 2023-12-30T00:00:00Z


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().